In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
dst_path = "/Users/sosen/UniProjects/eng-thesis/data/data-uncompressed/2D-tiff-grouped-random"

# Define parameters
batch_size = 32
img_height = 56
img_width = 56

In [ ]:

datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.2,
    # rotation_range=20,
    # zoom_range=0.15,
    # width_shift_range=0.2,
    # height_shift_range=0.2,
    # shear_range=0.15,
    # horizontal_flip=True,
    # fill_mode="nearest"
)

In [ ]:

# Generate training dataset
train_ds = datagen.flow_from_directory(
    directory=dst_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Generate validation dataset
val_ds = datagen.flow_from_directory(
    directory=dst_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
num_classes = len(val_ds.class_indices)


In [ ]:
x_sample, y_sample = next(train_ds)
print("Shape of input batch: ", x_sample.shape)
print("Shape of labels batch: ", y_sample.shape)

In [ ]:
from keras import backend as K

def f1(y_true, y_pred):
    def precision(y_true, y_pred):
        '''
        Precision metric. Only computes a batch-wise average of precision. Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        '''
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    def recall(y_true, y_pred):
        '''
        Recall metric. Only computes a batch-wise average of recall. Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        '''
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    y_pred = K.round(y_pred)
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:


base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(6, activation='softmax')(x)  # Replace num_classes with your actual number of classes

# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', f1])


In [ ]:
# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Assume you have validation data in val_ds
# Number of classes
# num_classes = len(val_ds.class_indices)

# Get true class labels
y_true = val_ds.classes

# Predict on validation data
predictions = model.predict(val_ds)

# Convert predictions to label index
y_pred = np.argmax(predictions, axis=1)

# Print classification report
labels = list(val_ds.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=labels))

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2)

# axs[0].plot(hi)
# Plot training & validation accuracy values
# axs[0].plot(history.history['accuracy'])
# axs[0].plot(history.history['val_accuracy'])
# axs[0].set_title('Model accuracy')
# axs[0].set_ylabel('Accuracy')
# axs[0].set_xlabel('Epoch')
# axs[0].legend(['Train', 'Val'], loc='upper left')
# 
# # Plot training & validation loss values
# axs[1].plot(history.history['loss'])
# axs[1].plot(history.history['val_loss'])
# axs[1].set_title('Model loss')
# axs[1].set_ylabel('Loss')
# axs[1].set_xlabel('Epoch')
# axs[1].legend(['Train', 'Val'], loc='upper left')

plt.show()